In [27]:
import numpy as np
import pandas as pd

In [28]:
df = pd.read_csv('Churn_Modelling.csv')
df

RowNumber  CustomerId    Surname  CreditScore Geography  Gender  Age  \
0             1    15634602   Hargrave          619    France  Female   42   
1             2    15647311       Hill          608     Spain  Female   41   
2             3    15619304       Onio          502    France  Female   42   
3             4    15701354       Boni          699    France  Female   39   
4             5    15737888   Mitchell          850     Spain  Female   43   
...         ...         ...        ...          ...       ...     ...  ...   
9995       9996    15606229   Obijiaku          771    France    Male   39   
9996       9997    15569892  Johnstone          516    France    Male   35   
9997       9998    15584532        Liu          709    France  Female   36   
9998       9999    15682355  Sabbatini          772   Germany    Male   42   
9999      10000    15628319     Walker          792    France  Female   28   

      Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0          2       0.00              1          1               1   
1          1   83807.86              1          0               1   
2          8  159660.80              3          1               0   
3          1       0.00              2          0               0   
4          2  125510.82              1          1               1   
...      ...        ...            ...        ...             ...   
9995       5       0.00              2          1               0   
9996      10   57369.61              1          1               1   
9997       7       0.00              1          0               1   
9998       3   75075.31              2          1               0   
9999       4  130142.79              1          1               0   

      EstimatedSalary  Exited  
0           101348.88       1  
1           112542.58       0  
2           113931.57       1  
3            93826.63       0  
4            79084.10       0  
...               ...     ...  
9995         96270.64       0  
9996        101699.77       0  
9997         42085.58       1  
9998         92888.52       1  
9999         38190.78       0  

[10000 rows x 14 columns]

In [29]:
def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))


In [30]:
# Convert categorical columns to numerical
df['Geography'] = df['Geography'].astype('category').cat.codes
df['Gender'] = df['Gender'].astype('category').cat.codes

# Drop irrelevant columns
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

# Splitting features and target
X = df.drop('Exited', axis=1).values.T
Y = df['Exited'].values.reshape(1, -1)

# Normalize the features
X = (X - X.mean()) / X.std()

# Constants
n_x = X.shape[0]  # Number of features
n_h = 4           # Number of units in hidden layer
n_y = 1           # Number of output units

# Train the neural network model
def neural_network_model(X, Y, n_h, num_iterations=10000, learning_rate=0.01):
    np.random.seed(3)
    n_x = X.shape[0]
    n_y = Y.shape[0]

    # Initialize parameters
    def initialize_parameters(n_x, n_h, n_y):
        np.random.seed(2)
        W1 = np.random.randn(n_h, n_x) * 0.01
        b1 = np.zeros((n_h, 1))
        W2 = np.random.randn(n_y, n_h) * 0.01
        b2 = np.zeros((n_y, 1))

        return {"W1": W1, "b1": b1, "W2": W2, "b2": b2}

    parameters = initialize_parameters(n_x, n_h, n_y)

    # Forward propagation
    def forward_propagation(X, parameters):
        W1 = parameters["W1"]
        b1 = parameters["b1"]
        W2 = parameters["W2"]
        b2 = parameters["b2"]

        Z1 = np.dot(W1, X) + b1
        A1 = np.tanh(Z1)
        Z2 = np.dot(W2, A1) + b2
        A2 = sigmoid(Z2)

        cache = {"Z1": Z1, "A1": A1, "Z2": Z2, "A2": A2}
        return A2, cache

    # Compute loss
    def compute_loss(A2, Y):
        m = Y.shape[1]
        logprobs = np.multiply(np.log(A2), Y) + np.multiply(np.log(1 - A2), (1 - Y))
        cost = - np.sum(logprobs) / m
        cost = np.squeeze(cost)
        return cost

    # Backward propagation
    def backward_propagation(parameters, cache, X, Y):
        m = X.shape[1]
        W1 = parameters["W1"]
        W2 = parameters["W2"]
        A1, A2 = cache['A1'], cache['A2']

        dZ2 = A2 - Y
        dW2 = (1 / m) * np.dot(dZ2, A1.T)
        db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
        dZ1 = np.dot(W2.T, dZ2) * (1 - np.power(A1, 2))
        dW1 = (1 / m) * np.dot(dZ1, X.T)
        db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)

        return {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}

    # Update parameters
    def update_parameters(parameters, grads, learning_rate):
        W1 = parameters["W1"]
        b1 = parameters["b1"]
        W2 = parameters["W2"]
        b2 = parameters["b2"]
        dW1 = grads["dW1"]
        db1 = grads["db1"]
        dW2 = grads["dW2"]
        db2 = grads["db2"]

        W1 -= learning_rate * dW1
        b1 -= learning_rate * db1
        W2 -= learning_rate * dW2
        b2 -= learning_rate * db2

        return {"W1": W1, "b1": b1, "W2": W2, "b2": b2}

    # Training loop
    for i in range(num_iterations):
        # Forward propagation
        A2, cache = forward_propagation(X, parameters)

        # Compute cost
        cost = compute_loss(A2, Y)

        # Backward propagation
        grads = backward_propagation(parameters, cache, X, Y)

        # Update parameters
        parameters = update_parameters(parameters, grads, learning_rate)

        # Print cost every 1000 iterations
        if i % 1000 == 0:
            print("Cost after iteration {}: {}".format(i, cost))

    return parameters

# Train the model
trained_parameters = neural_network_model(X, Y, n_h)

# Make predictions
def predict(parameters, X):
    A2, _ = forward_propagation(X, parameters)
    predictions = (A2 > 0.5).astype(int)
    return predictions

# Predict
predictions = predict(trained_parameters, X)

# Print predictions
print("Predictions:", predictions)

Cost after iteration 0: 0.6931616043163656
Cost after iteration 1000: 0.5081747715886612
Cost after iteration 2000: 0.5049192811301259
Cost after iteration 3000: 0.5025426702937251
Cost after iteration 4000: 0.500252282407833
Cost after iteration 5000: 0.49904849763939474
Cost after iteration 6000: 0.49853969540065657
Cost after iteration 7000: 0.49830891965886315
Cost after iteration 8000: 0.4981879855883573
Cost after iteration 9000: 0.4981149914899235
Predictions: [[0 0 0 ... 0 0 0]]
